## Imports and config (setting the random seed)

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib

pd.pandas.set_option('display.max_columns', None)
RANDOM_STATE = 42

## Train parameters to pickle

Our feature engineering techniques will learn:

* mean
* mode
* exponents for the yeo-johnson
* category frequency
* and category to number mappings

from the **train set.**

In [30]:
data = pd.read_csv('data/house_prices/train.csv') # reserve kaggle test for final evaluation
print(data.shape)

(1460, 81)


In [31]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Train-test split

In [32]:
# Let's separate into train and test set
# Remember to set the seed (random_state for this sklearn function)

X_train, X_valid, y_train, y_valid = train_test_split(
    data.drop(['Id', 'SalePrice'], axis=1), 
    data['SalePrice'], 
    test_size=0.1, 
    random_state=RANDOM_STATE,
)

X_train.shape, X_valid.shape

((1314, 79), (146, 79))

## Feature engineering: features

In the following cells, we will engineer the variables of the **House Price Dataset** so that we tackle:

- Missing values
- Temporal variables
- Non-Gaussian distributed variables
- Categorical variables: remove rare labels
- Categorical variables: convert strings to numbers
- Put the variables in a similar scale


## Feature engineering: target


In [33]:
y_train = np.log(y_train)
y_valid = np.log(y_valid)

## Missing values

1. Replace nans with string "missing" in vars with lots of missing data.
2. Replace nans with most frequent category in vars that contain fewer observations without values. 

In [34]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
cat_vars = cat_vars + ['MSSubClass']

X_train[cat_vars] = X_train[cat_vars].astype('O')
X_valid[cat_vars] = X_valid[cat_vars].astype('O')

In [35]:
len(cat_vars)

44

In [36]:
cat_vars_with_na = [
    var for var in cat_vars
    if X_train[var].isnull().sum() > 0
] # we do all feature engineering based on train set.

# print percentage of missing values per variable
X_train[cat_vars_with_na ].isnull().mean().sort_values(ascending=False)

PoolQC          0.994673
MiscFeature     0.961948
Alley           0.934551
Fence           0.808219
FireplaceQu     0.468798
GarageCond      0.055556
GarageQual      0.055556
GarageFinish    0.055556
GarageType      0.055556
BsmtExposure    0.025114
BsmtFinType2    0.024353
BsmtFinType1    0.024353
BsmtCond        0.024353
BsmtQual        0.024353
MasVnrType      0.006088
Electrical      0.000761
dtype: float64

In [37]:
# variables to impute with the string missing
with_string_missing = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() > 0.1]

# variables to impute with the most frequent category
with_frequent_category = [
    var for var in cat_vars_with_na if X_train[var].isnull().mean() < 0.1]


In [38]:
with_string_missing

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [39]:
with_frequent_category

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [40]:
X_train[with_string_missing] = X_train[with_string_missing].fillna('Missing')
X_valid[with_string_missing] = X_valid[with_string_missing].fillna('Missing')

In [41]:
for var in with_frequent_category:
    
    # there can be more than 1 mode in a variable; take first
    mode = X_train[var].mode().tolist()[0]
    
    print(var, mode)
    
    # replace nans with train mode
    X_train[var].fillna(mode, inplace=True)
    X_valid[var].fillna(mode, inplace=True)


MasVnrType None
BsmtQual TA
BsmtCond TA
BsmtExposure No
BsmtFinType1 Unf
BsmtFinType2 Unf
Electrical SBrkr
GarageType Attchd
GarageFinish Unf
GarageQual TA
GarageCond TA


In [42]:
X_train[cat_vars_with_na].isnull().sum().sum() # count all nans in any column

0

In [43]:
X_valid[cat_vars].isnull().sum().sum()

0

## Numerical variables

1. create indicator variable `{var}_na` for every numerical variable `var`.
2. fill na with mean. 

In [44]:
num_vars = [
    var for var in X_train.columns if var not in cat_vars and var != 'SalePrice'
]

# number of numerical variables
len(num_vars)

35

In [45]:
# make a list with the numerical variables that contain missing values
num_vars_with_na = [
    var for var in num_vars
    if X_train[var].isnull().sum() > 0
]

# print percentage of missing values per variable
X_train[num_vars_with_na].isnull().mean()

LotFrontage    0.180365
MasVnrArea     0.006088
GarageYrBlt    0.055556
dtype: float64

**Remark**: mean imputation with `GarageYrBlt` is weird. Perhaps better is DecisionTree imputation. 

In [46]:
# replace missing values as we described above
for var in num_vars_with_na:

    # calculate the mean using the train set
    mean_val = X_train[var].mean()
    
    print(var, mean_val)

    # add binary missing indicator (in train and test)
    X_train['{var}_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_valid['{var}_na'] = np.where(X_valid[var].isnull(), 1, 0)

    # replace missing values by the train mean
    X_train[var].fillna(mean_val, inplace=True)
    X_valid[var].fillna(mean_val, inplace=True)

LotFrontage 69.79387186629526
MasVnrArea 103.8009188361409
GarageYrBlt 1978.5729250604352


In [47]:
X_train[num_vars_with_na].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

In [48]:
X_valid[num_vars].isnull().sum().sum()

0

## Temporal variables

We learned in the previous notebook, that there are 4 variables that refer to the years in which the house or the garage were built or remodeled.

We will capture the time elapsed between those variables and the year in which the house was sold:

In [49]:
def elapsed_years(df, var):
    # capture difference between the year variable
    # and the year in which the house was sold
    df[var] = df['YrSold'] - df[var]
    return df

In [50]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    X_train = elapsed_years(X_train, var)
    X_valid = elapsed_years(X_valid, var)

In [51]:
# now we drop YrSold
X_train.drop(['YrSold'], axis=1, inplace=True)
X_valid.drop(['YrSold'], axis=1, inplace=True)

## Numerical variable transformation